In [727]:
from bs4 import BeautifulSoup
import urllib.request
import re
from datetime import datetime

import numpy as np
import pandas as pd

%load_ext autotime

import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


time: 7 ms


## Web Scraping

In [2]:
html_doc = "https://h1bdata.info/index.php?em=&job=Data+Scientist&city=&year=All+Years"
try:
    page = urllib.request.urlopen(html_doc)
except:
    print("Error loading the page.")

In [3]:
soup = BeautifulSoup(page, 'html.parser')

In [4]:
soup.title

<title>Data scientist Salary</title>

In [5]:
soup.title.name

'title'

In [6]:
soup.title.string


'Data scientist Salary'

In [7]:
soup.p

<p>Trending now:
<a href="index.php?em=Facebook&amp;year=2019">Facebook</a>, <a href="index.php?em=amazon&amp;year=2019">Amazon</a>, <a href="index.php?em=apple&amp;year=2019">Apple</a>,
<a href="index.php?em=netflix&amp;year=2019">Netflix</a>, <a href="index.php?em=google&amp;year=2019">Google</a>, <a href="index.php?em=airbnb&amp;year=2019">Airbnb</a>,
<a href="index.php?em=uber&amp;year=2019">Uber</a>, <a href="index.php?em=linkedin&amp;year=2019">Linkedin</a>, <a href="index.php?em=salesforce&amp;year=2019">Salesforce</a>
</p>

In [8]:
soup.a

<a class="navbar-brand" href="index.php">H1B Salary Database</a>

In [9]:
soup.find_all('a')[0]

<a class="navbar-brand" href="index.php">H1B Salary Database</a>

In [10]:
tables = soup.findChildren('table')
salary_table = tables[0]
rows = salary_table.findChildren(['tr'])

In [623]:
rows[0]

<tr><th>EMPLOYER</th><th>JOB TITLE</th><th>BASE SALARY</th><th>LOCATION</th><th data-date-format="mm/dd/yy">SUBMIT DATE</th><th data-date-format="mm/dd/yy">START DATE</th><th>CASE STATUS</th></tr>

In [639]:
# get column names (usage of list comprehension)
columns = [i.string for i in rows[0].findChildren(['th'])]

In [640]:
columns

['EMPLOYER',
 'JOB TITLE',
 'BASE SALARY',
 'LOCATION',
 'SUBMIT DATE',
 'START DATE',
 'CASE STATUS']

In [685]:
# store data in a list of lists
salaries_text = [[tr.text for tr in tr.find_all('td')] for tr in rows[1:]]

In [686]:
salaries_text[:3]

[['PERCOLATA CORPORATION',
  'DATA SCIENTIST',
  '46,060',
  'PALO ALTO, CA',
  '03/18/2016',
  '09/02/2016',
  'CERTIFIED'],
 ['MY LIFE REGISTRY LLC',
  'DATA SCIENTIST',
  '47,960',
  'FORT LEE, NJ',
  '02/18/2015',
  '08/20/2015',
  'CERTIFIED'],
 ['MY LIFE REGISTRY LLC',
  'DATA SCIENTIST',
  '47,960',
  'FORT LEE, NJ',
  '02/18/2015',
  '08/20/2015',
  'CERTIFIED']]

## Data Preprocessing

In [687]:
data_scientists = pd.DataFrame(salaries_text, columns=columns)

In [688]:
data_scientists.head()

,EMPLOYER,JOB TITLE,BASE SALARY,LOCATION,SUBMIT DATE,START DATE,CASE STATUS
0,PERCOLATA CORPORATION,DATA SCIENTIST,"46,060","PALO ALTO, CA",03/18/2016,09/02/2016,CERTIFIED
1,MY LIFE REGISTRY LLC,DATA SCIENTIST,"47,960","FORT LEE, NJ",02/18/2015,08/20/2015,CERTIFIED
2,MY LIFE REGISTRY LLC,DATA SCIENTIST,"47,960","FORT LEE, NJ",02/18/2015,08/20/2015,CERTIFIED
3,BIG DATA ANALYTICS TRADING INC,DATA SCIENTIST,"50,000","JOHNS CREEK, GA",03/16/2017,09/01/2017,CERTIFIED
4,YONO HEALTH INC,DATA SCIENTIST,"50,220","SUNNYVALE, CA",02/28/2017,08/28/2017,CERTIFIED


In [689]:
data_scientists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7344 entries, 0 to 7343
Data columns (total 7 columns):
EMPLOYER       7344 non-null object
JOB TITLE      7344 non-null object
BASE SALARY    7344 non-null object
LOCATION       7344 non-null object
SUBMIT DATE    7344 non-null object
START DATE     7344 non-null object
CASE STATUS    7344 non-null object
dtypes: object(7)
memory usage: 401.8+ KB


In [690]:
data_scientists['salary'] = data_scientists['BASE SALARY'].str.replace(',', '').astype(str).astype(int)

In [691]:
data_scientists['date'] = pd.to_datetime(data_scientists['SUBMIT DATE'])
data_scientists['year'] = data_scientists['date'].dt.year

In [692]:
data_scientists = data_scientists.rename(columns={'EMPLOYER': 'employer', 'JOB TITLE': 'job_title', 
                                                  'LOCATION': 'location', 'CASE STATUS': 'case_status'})

In [693]:
data_scientists = data_scientists[data_scientists['salary'] < 300000]
data_scientists = data_scientists[data_scientists['year'] > 2013]

In [849]:
data_scientists['state'] = data_scientists['location'].str.split(',', expand=True)[1]

time: 32 ms


In [694]:
del data_scientists['BASE SALARY']
del data_scientists['SUBMIT DATE']
del data_scientists['START DATE']

In [859]:
data_scientists.head()

,employer,job_title,location,case_status,salary,date,year,state
0,PERCOLATA CORPORATION,DATA SCIENTIST,"PALO ALTO, CA",CERTIFIED,46060,2016-03-18,2016,CA
1,MY LIFE REGISTRY LLC,DATA SCIENTIST,"FORT LEE, NJ",CERTIFIED,47960,2015-02-18,2015,NJ
2,MY LIFE REGISTRY LLC,DATA SCIENTIST,"FORT LEE, NJ",CERTIFIED,47960,2015-02-18,2015,NJ
3,BIG DATA ANALYTICS TRADING INC,DATA SCIENTIST,"JOHNS CREEK, GA",CERTIFIED,50000,2017-03-16,2017,GA
4,YONO HEALTH INC,DATA SCIENTIST,"SUNNYVALE, CA",CERTIFIED,50220,2017-02-28,2017,CA


time: 13 ms


## Hired Data Scientists over Time

In [696]:
count_hires = data_scientists.groupby('year')[['job_title']].count()
count_hires = count_hires.rename(columns = {'job_title': 'job_count'})
count_hires

,job_count
year,
2014,74
2015,561
2016,909
2017,1286
2018,2177
2019,2332


In [698]:
fig = go.Figure([go.Bar(
    x=count_hires.index, 
    y=count_hires.job_count, 
    text=count_hires.job_count, 
    textposition='auto')])
fig.update_layout(title = 'Data Science Hires over Years', xaxis_title = 'Year', yaxis_title = 'Count')
fig.show()

## Mean & Median Salaries over Years

In [700]:
agg_salaries = data_scientists.groupby('year').agg({'salary': [np.median, np.mean]})
agg_salaries.columns = ["_".join(x) for x in agg_salaries.columns.ravel()]
agg_salaries

,salary_median,salary_mean
year,,
2014,120000.0,121693.756757
2015,108597.0,106775.090909
2016,104021.0,104288.810781
2017,104749.0,105875.534992
2018,106000.0,109141.866330
2019,102865.5,109064.224700


In [701]:
fig = go.Figure(data=[go.Bar(
    name = 'Median',
    x=agg_salaries.index, 
    y=agg_salaries.salary_median),
                     go.Bar(
                         name = 'Mean',
                         x = agg_salaries.index,
                     y = agg_salaries.salary_mean)])
fig.update_layout(title = 'Mean & Median Salary over Years', xaxis_title = 'Year', yaxis_title = 'Salary in USD')
fig.show()

In [702]:
fig = go.Figure()
for i in data_scientists.year.unique():
    fig.add_trace(go.Box(y = data_scientists[data_scientists.year==i]['salary'],
                        name = str(i)))
fig.update_layout(title = 'Salary Box Plot over Years', xaxis_title = 'Year', yaxis_title = 'Salary in USD')
fig.show()

## Histogram

Let us check if the distribution of salaries approximates the famous normal distribution.

In [703]:
fig = go.Figure(data=[go.Histogram(x=data_scientists['salary'], histnorm='probability')])
fig.update_layout(title = 'Salary Histogram', xaxis_title = 'Salary Range in USD', yaxis_title = 'Count')
fig.show()

## Comparing Companies

In [721]:
companies = data_scientists.groupby('employer').agg({'salary': ['mean', 'median', 'min', 'max', 'count']})
companies.columns = ["_".join(x) for x in companies.columns.ravel()]
companies = companies[companies['salary_count'] > 10]
companies = companies.round({'salary_mean': 0})
companies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, ADOBE INC to ZILLOW INC
Data columns (total 5 columns):
salary_mean      70 non-null float64
salary_median    70 non-null float64
salary_min       70 non-null int32
salary_max       70 non-null int32
salary_count     70 non-null int64
dtypes: float64(2), int32(2), int64(1)
memory usage: 2.7+ KB


In [719]:
companies = companies.sort_values(by='salary_median', ascending=False)

data = [go.Bar(
    y=companies.index, 
    x=companies.salary_median, 
    text=companies.salary_median, 
    textposition='inside',
        textfont=dict(
        family="sans serif",
        size=100    ),
    orientation='h')]

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1700
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title = 'Median Salary by Company', 
    xaxis_title = 'Salary in USD',
    yaxis_dtick = 1
)
fig.show()

In [722]:
companies = companies.sort_values(by='salary_mean', ascending=False)
data = [go.Bar(
    y=companies.index, 
    x=companies.salary_mean, 
    text=companies.salary_mean, 
    textposition='inside',
        textfont=dict(
        family="sans serif",
        size=100    ),
    orientation='h')]

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1700
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title = 'Mean Salary by Company', 
    xaxis_title = 'Salary in USD',
    yaxis_dtick = 1
)
fig.show()

## Trend of Salaries by Company

In [725]:
all_emps = []
for i in data_scientists.employer.unique():
    temp = data_scientists[data_scientists['employer'] == i].groupby('year').agg({'salary': ['mean', 'median', 'min', 'max','count']})
    temp['employer'] = i
    all_emps.append(temp)

In [779]:
employers = pd.concat(all_emps)

time: 2.98 s


In [780]:
employers.columns = ["_".join(x) for x in employers.columns.ravel()]
employers = employers.rename(columns={'employer_':'employer'})

time: 13 ms


In [813]:
fig = go.Figure()
for i in companies.index.unique():
    temp = employers[employers['employer'] == i]
    fig.add_trace(go.Scatter(x=temp.index, y=temp['salary_mean'], name=i))
fig.show()

time: 1.29 s


## Location-based Analytics

### Comparing Cities

In [826]:
loc_salaries = data_scientists.groupby('location').agg({'salary': ['median', 'mean', 'count']})
loc_salaries.columns = ["_".join(x) for x in loc_salaries.columns.ravel()]
loc_salaries = loc_salaries[loc_salaries['salary_count'] > 20]
loc_salaries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, ALPHARETTA, GA to WASHINGTON, DC
Data columns (total 3 columns):
salary_median    44 non-null float64
salary_mean      44 non-null float64
salary_count     44 non-null int64
dtypes: float64(2), int64(1)
memory usage: 1.4+ KB
time: 41 ms


In [830]:
loc_salaries = loc_salaries.sort_values(by='salary_median', ascending=False)

data = [go.Bar(
    y=loc_salaries.index, 
    x=loc_salaries.salary_median, 
    text=loc_salaries.salary_median, 
    textposition='inside',
        textfont=dict(
        family="sans serif",
        size=100),
    orientation='h')]

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1200
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title = 'Median Salary by Location', 
    xaxis_title = 'Salary in USD',
    yaxis_dtick = 1
)
fig.show()

time: 330 ms


### Comparing States

In [851]:
state_salaries = data_scientists.groupby('state').agg({'salary': ['median', 'mean', 'count']})
state_salaries.columns = ["_".join(x) for x in state_salaries.columns.ravel()]
state_salaries.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries,  to  WY
Data columns (total 3 columns):
salary_median    47 non-null float64
salary_mean      47 non-null float64
salary_count     47 non-null int64
dtypes: float64(2), int64(1)
memory usage: 1.5+ KB
time: 200 ms


In [856]:
state_salaries = state_salaries.sort_values(by='salary_count', ascending=False)

data = [go.Bar(
    x=state_salaries.index, 
    y=state_salaries.salary_count, 
    text=state_salaries.salary_count, 
)]

layout = go.Layout(
    autosize=False,
    width=1000,
    height=500
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title = 'Hires by State', 
    xaxis_title = 'State',
    yaxis_title = 'Hires')
fig.show()

time: 253 ms


In [857]:
state_salaries = state_salaries.sort_values(by='salary_median', ascending=False)

data = [go.Bar(
    x=state_salaries.index, 
    y=state_salaries.salary_median, 
    text=state_salaries.salary_median, 
)]

layout = go.Layout(
    autosize=False,
    width=1000,
    height=500
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title = 'Median Salary by State', 
    xaxis_title = 'State',
    yaxis_title = 'Salary')
fig.show()

time: 239 ms
